In [1]:
import numpy as np
from collections import defaultdict
from abc import ABC, abstractmethod
import time

## Monte Carlo Tree Search

Find concept explanaition [here.](https://docs.google.com/presentation/d/1pUfIsVIqDjR9MOxdywbf2ewMiwJw65TdEamWOdncZRA/edit?usp=sharing)

[TicTacToe Online](https://www.cbc.ca/kids/games/play/ultimate-tic-tac-toe)

## Task
- As usual, fill in the gaps in the code.
- Your goal is to select a move at each step for tic tac toe game using MCTS.
- You can simulate games with random moves.

First, define the TicTacToeMove object, it will represent player's move.

In [9]:
class TicTacToeMove:
    def __init__(self, x_coordinate, y_coordinate, value):
        """
        Parameters
        ----------
        x_coordinate : int
        y_coordinate: int
        value : value to put in the required cell, 1 for x, -1 for 0
        """
        self.x_coordinate = x_coordinate
        self.y_coordinate = y_coordinate
        self.value = value

Next, define TicTacToeGameState, it will represent game state at a time.

It will contain the following knowledge:

- board
- player who is going to make next move

It can also provide the following information about game:

- is_game_over
- game_result - return result of the gave if over
- move - performs TicTacToeMove, put 0 or x to a given position
- is_move_legal - checks wheather a move is allowed to make, e.g. cell is empty
- get_legal_actions - returns empty cells where value could be placed

In [10]:
class TicTacToeGameState:

    x = 1
    o = -1

    def __init__(self, board, next_to_move=1):
        """
        Parameters
        ----------
        state : Array(3,3) filled with 0, 1, -1. Current state of the board
        next_to_move: who is going to make next move?
        """

        if len(board.shape) != 2 or board.shape[0] != board.shape[1]:
            raise ValueError("Only 2D square boards allowed")
        self.board = board
        self.board_size = board.shape[0]
        self.next_to_move = next_to_move

    def game_result(self):
        """
        Returns
        -------
        Int or None 1 for x win, 0 for tie, -1 for 0 win
        """
        # check if game is over
        rowsum = np.sum(self.board, axis=1)
        colsum = np.sum(self.board, axis=0)
        diag_sum_tl = np.sum(np.diag(self.board))
        diag_sum_tr = np.sum(np.diag(np.fliplr(self.board)))

        player_one_wins = any(rowsum == self.board_size)
        player_one_wins += any(colsum == self.board_size)
        player_one_wins += (diag_sum_tl == self.board_size)
        player_one_wins += (diag_sum_tr == self.board_size)

        if player_one_wins:
            return self.x

        player_two_wins = any(rowsum == -self.board_size)
        player_two_wins += any(colsum == -self.board_size)
        player_two_wins += (diag_sum_tl == -self.board_size)
        player_two_wins += (diag_sum_tr == -self.board_size)

        if player_two_wins:
            return self.o

        # todo: do not forget to check for tie
        if not np.any(self.board == 0):
            return 0

        # if not over - no result
        return None

    def is_game_over(self):
        """
        Returns
        -------
        Boolean
        """
        return self.game_result() is not None

    def is_move_legal(self, move):
        """
        Returns
        -------
        Boolean
        """
        if self.board[move.x_coordinate, move.y_coordinate] != 0:
            return False
        if move.x_coordinate < 0 or move.x_coordinate >= self.board_size:
            return False
        if move.y_coordinate < 0 or move.y_coordinate >= self.board_size:
            return False
        return True

    def move(self, move):
        """
        Perform actual move, e.g. fill in the corresponding point with given
        value. Do not forget to change next to move person.

        Parameters
        ----------
        move : TicTacToeMove

        Returns
        -------
        new TicTacToeGameState
        """
        if not self.is_move_legal(move):
            raise ValueError(  f"move {move} on board {self.board} is not legal" )
        new_board = np.copy(self.board)

        new_board[move.x_coordinate, move.y_coordinate] = move.value
        return TicTacToeGameState(new_board, next_to_move=-self.next_to_move)

    def get_legal_actions(self):
        """
        Returns list of available coordinates for next move
        return: List[TicTacToeMove]
        """
        indices = np.where(self.board == 0)
        # ???
        # todo: convert this into list[TicTacToeMove]
        return [
            TicTacToeMove(coords[0], coords[1], self.next_to_move)
            for coords in list(zip(indices[0], indices[1]))
        ]

Next, we need to define a node of MCTS, in MonteCarloTreeSearchNode.


In [11]:
class MonteCarloTreeSearchNode:

    def __init__(self, state, parent=None):
        """
        Parameters
        ----------
        state : TicTacToeGameState
        parent : MonteCarloTreeSearchNode
        children : List[MonteCarloTreeSearchNode]
        number_of_visits: number of time this node was simulated
        """
        self.state = state
        self.parent = parent
        self.children = []
        self.number_of_visits = 0.
        self._results = defaultdict(int) # results of the simulated game, 1 if x won, -1 if 0 won, 0 for tie
        self._untried_actions = None # List[TicTacToeMove] empty cells allowed to move

    def untried_actions(self):
        """
        Returns
        -------
        list of TicTacToeMove
        """
        if self._untried_actions is None:
            self._untried_actions = self.state.get_legal_actions()
        return self._untried_actions

    def wins(self):
        """
        Returns:
        -------
        int, number of wins
        """
        return self._results[1] - self._results[-1]

    def expand(self):
        """
        Makes a move with one of the untried actions

        Returns:
        ------
        MonteCarloTreeSearchNode
        """
        action = self.untried_actions().pop()
        next_state = self.state.move(action)
        child_node = MonteCarloTreeSearchNode(next_state, parent=self)
        self.children.append(child_node)
        return child_node

    def is_terminal_node(self):
        return self.state.is_game_over()

    def rollout(self):
        """
        Plays random game from a given state

        Returns:
        ------
        TicTacToeGameState.game_result(), e.g. -1 for 0 win, 1 for x win, 0 for tie, None for not finished game
        """
        # Utilize get_legal_actions here and move
        current_rollout_state = self.state
        while not current_rollout_state.is_game_over():
            possible_moves = current_rollout_state.get_legal_actions()
            action = possible_moves[np.random.randint(len(possible_moves))]
            current_rollout_state = current_rollout_state.move(action)
        return current_rollout_state.game_result()

    def backpropagate(self, result):
        """
        Increase number of games we have played
        In case of win, increse number of wins
        Propagate results to parents

        Parameters
        ----------
        result : 1 if x won, -1 if 0 won, 0 for tie
        """

        self.number_of_visits += 1.
        self._results[result] += 1.

        if self.parent:
            self.parent.backpropagate(result)

    def is_fully_expanded(self):
        """
        Are there any more moves can be performed?
        """
        return len(self.untried_actions()) == 0

    def best_child(self, c_param=1.4):
        """
        Here we are choosing which child we are going to expand next,
        according to formula from the slides.

        Parameters:
        ------
        c_param: float - exploration vs exploitation trade off

        """
        choices_weights = [
            (child.wins() / child.number_of_visits) + c_param * np.sqrt((2 * np.log(self.number_of_visits) / child.number_of_visits))
            for child in self.children
        ]
        return self.children[np.argmax(choices_weights)]

In [12]:
class MonteCarloTreeSearch(object):

    def __init__(self, node):
        """
        MonteCarloTreeSearchNode
        Parameters
        ----------
        node : MonteCarloTreeSearchNode
        """
        self.root = node

    def best_action(self, simulations_number=None):
        """
        Parameters
        ----------
        simulations_number : int - number of simulations performed to get the best action
        Returns
        -------
        """
        for _ in range(0, simulations_number):
            v = self._tree_policy()
            reward = v.rollout()
            v.backpropagate(reward)
        # to select best child go for exploitation only
        return self.root.best_child(c_param=0.)

    def _tree_policy(self):
        """
        selects node to run rollout/playout for
        Returns
        -------
        """
        current_node = self.root
        while not current_node.is_terminal_node():
            if not current_node.is_fully_expanded():
                return current_node.expand()
            else:
                current_node = current_node.best_child()
        return current_node

In [13]:
def print_tic_tac_toe(values):
    def get_print_value(value):
      if value == -1:
        return '0'
      elif value == 0:
        return ' '
      else:
       return 'x'

    print("\n")
    print("\t     |     |")
    print("\t  {}  |  {}  |  {}".format(get_print_value(values[0][0]), get_print_value(values[0][1]), get_print_value(values[0][2])))
    print('\t_____|_____|_____')

    print("\t     |     |")
    print("\t  {}  |  {}  |  {}".format(get_print_value(values[1][0]), get_print_value(values[1][1]), get_print_value(values[1][2])))
    print('\t_____|_____|_____')

    print("\t     |     |")

    print("\t  {}  |  {}  |  {}".format(get_print_value(values[2][0]), get_print_value(values[2][1]), get_print_value(values[2][2])))
    print("\t     |     |")
    print("\n")

In [16]:
x = 1
o = -1


def game_step(board_state):
  root = MonteCarloTreeSearchNode(state = board_state)
  mcts = MonteCarloTreeSearch(root)
  best_node = mcts.best_action(100)

  print_tic_tac_toe(best_node.state.board)

  return best_node.state.board


board_state = TicTacToeGameState(board = np.zeros((3,3)), next_to_move=x)
while(not board_state.is_game_over()):
  board = game_step(board_state)
  board_state = TicTacToeGameState(board = board, next_to_move=board_state.next_to_move*-1)



	     |     |
	     |     |   
	_____|_____|_____
	     |     |
	     |     |   
	_____|_____|_____
	     |     |
	  x  |     |   
	     |     |




	     |     |
	     |  0  |   
	_____|_____|_____
	     |     |
	     |     |   
	_____|_____|_____
	     |     |
	  x  |     |   
	     |     |




	     |     |
	     |  0  |   
	_____|_____|_____
	     |     |
	     |     |   
	_____|_____|_____
	     |     |
	  x  |     |  x
	     |     |




	     |     |
	     |  0  |   
	_____|_____|_____
	     |     |
	     |     |  0
	_____|_____|_____
	     |     |
	  x  |     |  x
	     |     |




	     |     |
	     |  0  |   
	_____|_____|_____
	     |     |
	     |     |  0
	_____|_____|_____
	     |     |
	  x  |  x  |  x
	     |     |


